In [1]:
import pandas as pd
import numpy as np
import json
import datetime
import os
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/zips_2_DMAs_Kaylar_Bruce_20190613'

In [2]:
open_stores_as_20190608=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-06-08.csv",
                                   dtype=str)
max_date=open_stores_as_20190608['week_end_date'].max()
print(max_date)
open_stores_as_20190608=open_stores_as_20190608[open_stores_as_20190608['week_end_date']==max_date]
open_stores_as_20190608.shape

open_stores_as_20190608['sales']=open_stores_as_20190608['sales'].astype(float)
open_stores_as_20190608=open_stores_as_20190608[open_stores_as_20190608['sales']>0]

open_stores_as_20190608=open_stores_as_20190608[['location_id']]
open_stores_as_20190608.shape


2019-06-08


(1406, 1)

In [3]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190601-134302-813.txt",sep="|",dtype=str)
store_list=store_list[['location_id','zip_cd','latitude_meas','longitude_meas']]
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)

store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.split("-")[0])

open_stores_as_20190608=pd.merge(open_stores_as_20190608,store_list,on="location_id",how="left")
open_stores_as_20190608.shape

(1406, 4)

In [4]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190601-134302-813.txt",sep="|",dtype=str)
store_list.head(2)

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
0,3,PEACH ORCHARD PLAZA,1988-10-31,NaN,2708 PEACH ORCHARD RD,NaN,AUGUSTA,GA,30906-2408,-000000000082.018921,000000000033.421570,00000000026501.0000,00000000005098.0000
1,30,SOUTHWOOD PLZ S/C,1984-04-05,NaN,818 S MAIN ST,NaN,BOWLING GREEN,OH,43402-4601,-000000000083.650909,000000000041.363132,00000000021322.0000,00000000003271.0000


In [5]:
print(open_stores_as_20190608[pd.isnull(open_stores_as_20190608['zip_cd'])])
store_zip_missing_dict={"5396":"21703"}

     location_id zip_cd  latitude_meas  longitude_meas
1405        5396    NaN            NaN             NaN


In [6]:
open_stores_as_20190608_0=open_stores_as_20190608[pd.isnull(open_stores_as_20190608['zip_cd'])]
open_stores_as_20190608_0['zip_cd']=open_stores_as_20190608_0['location_id'].apply(lambda x: store_zip_missing_dict[x])
open_stores_as_20190608_1=open_stores_as_20190608[~pd.isnull(open_stores_as_20190608['zip_cd'])]
open_stores_as_20190608=open_stores_as_20190608_1.append(open_stores_as_20190608_0)

open_stores_as_20190608=open_stores_as_20190608.sort_values("location_id")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
DMA_nielsen=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1,usecols=[0,2])
DMA_nielsen=DMA_nielsen.drop_duplicates()
DMA_nielsen.columns=['zip_cd',"DMA"]

In [8]:
open_stores_as_20190608_DMA=pd.merge(open_stores_as_20190608,DMA_nielsen,on="zip_cd",how="left")

In [9]:
open_stores_as_20190608_2_target_DMA=open_stores_as_20190608_DMA[open_stores_as_20190608_DMA['DMA'].isin(["DENVER","CHARLESTON, SC"])]
print(open_stores_as_20190608_2_target_DMA.shape)

print(open_stores_as_20190608_DMA[open_stores_as_20190608_DMA['location_id'].isin(open_stores_as_20190608_2_target_DMA['location_id'].tolist())].shape)

# No store accross DMAs for the 2 targeted

(18, 5)
(18, 5)


In [10]:
store_labels=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",
                         dtype=str,usecols=['location_id','revenue_flag','zip'])

store_labels=store_labels[store_labels['revenue_flag']!="T"]
output_overall=pd.merge(open_stores_as_20190608_2_target_DMA,store_labels,on="location_id",how="left")
output_overall=output_overall.rename(columns={"zip_cd":"store_zip"})

In [11]:
output_overall_0=output_overall[pd.isnull(output_overall['revenue_flag'])]
print("output_overall_0",output_overall_0.shape)

output_overall_1=output_overall[~pd.isnull(output_overall['revenue_flag'])]
print("output_overall_1['location_id'].nunique()",output_overall_1['location_id'].nunique())


output_overall_0 (2, 7)
output_overall_1['location_id'].nunique() 16


In [12]:
output_overall_1.head(2)

,location_id,store_zip,latitude_meas,longitude_meas,DMA,zip,revenue_flag
0,117,29407,32.814445,-79.997371,"CHARLESTON, SC",29407,P
1,117,29407,32.814445,-79.997371,"CHARLESTON, SC",29414,P


In [13]:
output_overall_1=output_overall_1.reset_index()
del output_overall_1['index']
output_overall_1['distance_miles']=np.nan

for ind,row in output_overall_1.iterrows():
    dist=haversine((row['latitude_meas'],row['longitude_meas']),zip_centers[row['zip']],miles=True)
    output_overall_1.loc[ind,'distance_miles']=dist

output_overall_1=output_overall_1[(output_overall_1['revenue_flag']=="P") | (output_overall_1['distance_miles']<=20)]

In [14]:
output_overall_0=output_overall_0.reset_index()
del output_overall_0['index']

del output_overall_0['zip']
del output_overall_0['revenue_flag']
i_counter=0
df_new_store_zips=pd.DataFrame(columns=["location_id","zip","revenue_flag","distance_miles"])
for ind,row in output_overall_0.iterrows():
    store_number=row['location_id']
    store_center=[row['latitude_meas'],row['longitude_meas']]
    for zip_cd in zip_centers.keys():
        dist=haversine(zip_centers[zip_cd],store_center,miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":store_number,"zip":zip_cd,"revenue_flag":"zip_in_10","distance_miles":dist},index=[i_counter])
            i_counter+=1
            df_new_store_zips=df_new_store_zips.append(df)
    
output_overall_0=pd.merge(output_overall_0,df_new_store_zips,on="location_id",how="left")
    

In [16]:
cols=output_overall_1.columns.tolist()
output_overall=output_overall_0.append(output_overall_1)[cols]

writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/zips_2_DMAs_Kaylar_Bruce_20190613/BL_zips_for_stores_in_2_test_DMAs_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

for DMA,group in output_overall.groupby('DMA'):
    unique_zips_DMA=group[['zip']].drop_duplicates()
    unique_zips_DMA.to_excel(writer,"unique_zips_"+DMA,index=False)
    group.to_excel(writer,DMA,index=False)
writer.save()    